Embeddings with RNN (GRU).

In [595]:
import collections
import os
from collections import Counter
import numpy as np
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import string
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

from argparse import Namespace

In [596]:
args = Namespace(
    train_dataset_csv="train.csv",
    test_dataset_csv="test.csv",
    train_proportion=0.7,
    val_proportion=0.3,
    train_split_csv="train_with_splits.csv", 
    #train_split_csv="train_tweet.csv", 
    glove_patch='C:/Users/Admin/Desktop/NLP/embed/glove.6B.100d.txt',
    seed=123)

            

In [597]:
train_splits=pd.read_csv(args.train_split_csv)
train_splits

,id,keyword,location,text,target,split
0,9815,trauma,LOCAL ATLANTA NEWS 4/28/00 - 4/28/15 FREELANCER,@raabchar_9 @drphil @morganlawgrp how do you s...,0,train
1,4527,emergency,"Anchorage, AK",anchorage jobs emergency medicine - nurse prac...,0,train
2,6540,injury,NaN,joboozoso : usat usatoday_nfl michael floyd '...,0,train
3,1456,body bagging,NaN,wwe 9k9 mycareer ep9 tyrone body bagging dudes...,0,train
4,10566,windstorm,she/her/your majesty/empress,i like the weird ones like rain of mystical or...,0,train
...,...,...,...,...,...,...
7478,4742,evacuate,NaN,sooo police dispatch said there was a person t...,1,val
7479,9206,suicide bombing,NaN,suicide bombing is just the fear of dying alone,1,val
7480,2540,collision,"North Highlands, CA",traffic collision - ambulance enroute : elkho...,1,val
7481,4747,evacuate,Nashville,@ahhtheenikki and from what i can tell- they r...,1,val


In [598]:
train_splits['keyword']=train_splits.keyword.fillna('_nan_')

In [599]:
train=train_splits[train_splits.split=='train']
val=train_splits[train_splits.split=='val']
print(len(train), len(val))

5251 2232


The Vocabulary.

In [600]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
            
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index
        
        Args: 
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
   
    def __len__(self):
        return len(self._token_to_idx)

For text: Adding tokens of the beginning, end, unknown word, and the phrase complement symbol to the maximum length (mask).

In [601]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            'unk_index' needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

The Vectorizer.

In [602]:
class TweetVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""    
    def __init__(self, text_vocab, keyword_vocab):
        self.text_vocab = text_vocab
        self.keyword_vocab = keyword_vocab
        
    def vectorize_text(self, text, vector_length=-1):
        """
        Args:
            text (str): the string of words separated by a space
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vetorized text (numpy.array)
        """
        indices = [self.text_vocab.begin_seq_index]
        for token in text.split(" "):
            if (token not in string.punctuation):
                indices.append(self.text_vocab.lookup_token(token))
                
        indices.append(self.text_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.text_vocab.mask_index

        return out_vector
    
    def vectorize_keyword(self, keyword):
        indices=[]
        for word in keyword.split(' '):
            if (word not in string.punctuation):
                indices.append(self.keyword_vocab.lookup_token(word))
                
        out_vector=np.zeros(3, dtype=np.int64)
        out_vector[:len(indices)]=indices  
        
        return out_vector
     
    @classmethod
    def from_dataframe(cls, tweet_df, cutoff=3):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            tweet_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary 
        Returns:
            an instance of the TweetVectorizer
        """
        keyword_vocab = Vocabulary()        
        for keyword in tweet_df.keyword:
            for word in keyword.split(' '):
                keyword_vocab.add_token(word)

        word_counts = Counter()
        for text in tweet_df.text:            
            for token in text.split(" "):
                if (token not in string.punctuation):
                    #token_clear = re.sub(r"([\W9]+)", r"*", token)
                    #word_counts[token_clear] += 1
                     word_counts[token] += 1
        
        text_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                text_vocab.add_token(word)
        
        return cls(text_vocab, keyword_vocab)

    @classmethod
    def from_serializable(cls, contents):
        text_vocab = \
            SequenceVocabulary.from_serializable(contents['text_vocab'])
        keyword_vocab =  \
            Vocabulary.from_serializable(contents['keyword_vocab'])

        return cls(text_vocab=text_vocab, keyword_vocab=keyword_vocab)

    def to_serializable(self):
        return {'text_vocab': self.text_vocab.to_serializable(),
                'keyword_vocab': self.keyword_vocab.to_serializable()}

In [603]:
test_vect=TweetVectorizer.from_dataframe(train_splits)
test_vect.to_serializable()

{'text_vocab': {'token_to_idx': {'<MASK>': 0,
   '<UNK>': 1,
   '<BEGIN>': 2,
   '<END>': 3,
   'how': 4,
   'do': 5,
   'you': 6,
   'a': 7,
   'wound': 8,
   'to': 9,
   'your': 10,
   'side': 11,
   'and': 12,
   'force': 13,
   'trauma': 14,
   'not': 15,
   'with': 16,
   'fall': 17,
   'anchorage': 18,
   'jobs': 19,
   'emergency': 20,
   'nurse': 21,
   'healthcare': 22,
   'ak': 23,
   'em': 24,
   'http': 25,
   't': 26,
   'co': 27,
   'michael': 28,
   'floyd': 29,
   's': 30,
   'hand': 31,
   'injury': 32,
   'shouldn': 33,
   'devalue': 34,
   'his': 35,
   'fantasy': 36,
   'stock': 37,
   '9k9': 38,
   'ep9': 39,
   'body': 40,
   'bagging': 41,
   'dudes': 42,
   'via': 43,
   '@youtube': 44,
   'i': 45,
   'like': 46,
   'the': 47,
   'weird': 48,
   'ones': 49,
   'rain': 50,
   'of': 51,
   'or': 52,
   'windstorm': 53,
   'ocean': 54,
   'waves': 55,
   'https': 56,
   'love': 57,
   'amp': 58,
   'can': 59,
   'wait': 60,
   'until': 61,
   'collide': 62,
   'm':

The Dataset.

In [604]:
class TweetDataset(Dataset):
    def __init__(self, tweet_df, vectorizer):
        """
        Args:
            tweet_df (pandas.DataFrame): the dataset
            vectorizer (TweetVectorizer): vectorizer instatiated from dataset
        """
        self.tweet_df = tweet_df
        self._vectorizer = vectorizer

        # +1 if only using begin_seq, +2 if using both begin and end seq tokens
        def measure_len(text):
            len_=0
            for word in text:
                if word not in string.punctuation:
                    len_+=1
            return len_
                    
        #measure_len = lambda context: len(context.split(" "))
        self._max_seq_length = max(map(measure_len, tweet_df.text)) + 2
        

        self.train_df = self.tweet_df[self.tweet_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.tweet_df[self.tweet_df.split=='val']
        self.validation_size = len(self.val_df)

        
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size)}

        self.set_split('train')

        # Class weights
        class_counts = tweet_df.target.value_counts().to_dict()
        freq = [count for _, count in class_counts.items()]
        self.class_weights = 1.0 / torch.tensor(freq, dtype=torch.float32)
        
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, tweet_csv):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            tweet_csv (str): location of the dataset
        Returns:
            an instance of TweetDataset
        """
        tweet_df = pd.read_csv(tweet_csv)
        train_tweet_df = tweet_df[tweet_df.split=='train']
        return cls(tweet_df, TweetVectorizer.from_dataframe(train_tweet_df))

    
    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        text_vector = \
            self._vectorizer.vectorize_text(row.text, self._max_seq_length)

        keyword_vector = \
            self._vectorizer.vectorize_keyword(row.keyword)
        
        
        return {'x_data': text_vector,
                'keyword_vector': keyword_vector,                
                'y_target': row.target}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size


In [605]:
data=TweetDataset.load_dataset_and_make_vectorizer(args.train_split_csv)

In [606]:
data.__getitem__(78)

{'x_data': array([  2,   1, 515, 516, 517,  42,  87, 518,   7, 476, 519, 520, 193,
        376, 521,   8,  79,   7, 522,  48, 523, 462,  61, 524,  29, 263,
          3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0

In [607]:
train_splits.iloc[236].keyword, train_splits.iloc[236].text

('body bags',
 " ' your body will heal the bags under your eyes will go away you\x89ûªll be so happy you\x89ûªll smile and really .  .  .  '  http :  /  / t . co / wukcalnqms")

In [608]:
train_splits[train_splits.keyword=='body bags'].head(5)

,id,keyword,location,text,target,split
55,1484,body bags,United Kingdom,womens buckle casual stylish shoulder handbags...,0,train
68,1526,body bags,Speaking the Truth in Love,fairfax investigating firefighter over faceboo...,0,train
236,1519,body bags,NaN,' your body will heal the bags under your eye...,0,train
348,1520,body bags,"California, USA",womens handbags cross body geometric pattern s...,0,train
586,1486,body bags,CA,bestseller ! fossil dawson mini cross body ba...,0,train


As we can see, the keyword often does not reflect the context of the message at all.

In [609]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    """A generator function which wraps the PyTorch DataLoader. 
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

The Model: RNNClassifier.

In [610]:
class RNNClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, rnn_hidden_size, 
                 dropout_p, pretrained_embeddings=None, padding_idx=0):
        """
        Args:
            embedding_size (int): size of the embedding vectors (the size of each embedding vector)
            num_embeddings (int): number of embedding vectors(size of the dictionary of embeddings)            
            rnn_hidden_size (int): the size of the hidden dimension in GRU            
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings
                default is None. If provided, 
            padding_idx (int): an index representing a null position
        """
        super(RNNClassifier, self).__init__()

        if pretrained_embeddings is None:

            self.emb = nn.Embedding(num_embeddings=num_embeddings,
                                    embedding_dim=embedding_size,                                    
                                    padding_idx=padding_idx)        
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.emb = nn.Embedding(num_embeddings=num_embeddings,
                                    embedding_dim=embedding_size,                                    
                                    padding_idx=padding_idx,
                                    _weight=pretrained_embeddings)
            
        self.rnn = nn.GRU(input_size=embedding_size, hidden_size=rnn_hidden_size, batch_first=True, bidirectional=True)            
        self.fc = nn.Linear(in_features=rnn_hidden_size*2, out_features=1)
        self._dropout_p = dropout_p
        #self._relu = nn.ReLU()
        
    def forward(self, x_in, apply_sigmoid=False):
        
        x_embedded = self.emb(x_in)
        y_all, y_end = self.rnn(x_embedded)
        batch_size, seq_size, feat_size = y_all.shape
        y_all = y_all.contiguous().view(batch_size * seq_size, feat_size)
        #y_all=self._relu(y_all)
        y_all = self.fc(F.dropout(y_all, p=self._dropout_p))        
                               
        if apply_sigmoid:
            y_all = F.sigmoid(y_all)
            
        y_all = y_all.contiguous().view(batch_size, 1, seq_size)
        
        y_pred=F.avg_pool1d(y_all, seq_size).squeeze(dim=2)        
        
       
        return y_pred.squeeze()

In [611]:
import io

In [612]:
def load_glove_from_file(glove_filepath):
    """
    Load the GloVe embeddings 
    
    Args:
        glove_filepath (str): path to the glove embeddings file 
    Returns:
        word_to_index (dict), embeddings (numpy.ndarary)
    """

    word_to_index = {}
    embeddings = []
    #with open(glove_filepath, "r") as fp:
    with io.open(glove_filepath, encoding='utf-8') as fp:
        for index, line in enumerate(fp):
            line = line.split(" ") # each line: word num1 num2 ...
            word_to_index[line[0]] = index # word = line[0] 
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    return word_to_index, np.stack(embeddings)

def make_embedding_matrix(glove_filepath, words):
    """
    Create embedding matrix for a specific set of words.
    
    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataset
    """
    word_to_idx, glove_embeddings = load_glove_from_file(glove_filepath)
    embedding_size = glove_embeddings.shape[1]
    
    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = glove_embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

In [613]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_score': [],
            'val_loss': [],
            'val_score': [],
            'test_loss': -1,
            'test_score': -1,
            'model_filename': args.model_state_file}


Settings and some prep work.

In [614]:
args = Namespace(
    train_dataset_csv="train.csv",
    test_dataset_csv="test.csv",
    train_proportion=0.7,
    val_proportion=0.3,
    train_split_csv="train_with_splits.csv", 
    #train_split_csv="train_tweet.csv",
    glove_patch='C:/Users/Admin/Desktop/NLP/embed/glove.6B.100d.txt',
    model_state_file="model.pth",
    seed=123,
    use_glove=False,
    embedding_size=100, 
    rnn_hidden_size=100,     
    # Training hyper parameter    
    learning_rate=0.001, 
    dropout_p=0.3, 
    batch_size=128, 
    num_epochs=30, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True
   
) 



In [615]:
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False
    
args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

Using CUDA: True


Initializations.

In [616]:
args.use_glove = True

In [617]:
# create dataset and vectorizer
dataset = TweetDataset.load_dataset_and_make_vectorizer(args.train_split_csv)

vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.text_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(glove_filepath=args.glove_patch, 
                                       words=words)
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None

Using pre-trained embeddings


In [618]:
embeddings.shape

(3404, 100)

In [619]:
classifier = RNNClassifier(embedding_size=args.embedding_size, 
                            num_embeddings=len(vectorizer.text_vocab),                            
                            rnn_hidden_size=args.rnn_hidden_size,                            
                            dropout_p=args.dropout_p,
                            pretrained_embeddings=embeddings,
                            #pretrained_embeddings=None,
                            padding_idx=0)
classifier

RNNClassifier(
  (emb): Embedding(3404, 100, padding_idx=0)
  (rnn): GRU(100, 100, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=200, out_features=1, bias=True)
)

In [620]:
def compute_score(y_target, y_pred):
    y_target_nump = y_target.cpu().bool().numpy()
    y_pred_nump = (torch.sigmoid(y_pred)>0.5).cpu().numpy()
    #y_pred_nump = (y_pred>0.5).cpu().long().numpy()
    if y_target_nump.sum()==0:
        zv=1
    else:
        zv=0   
    prec=precision(y_target_nump, y_pred_nump, zero_division=zv)
    rec=recall(y_target_nump, y_pred_nump, zero_division=zv) 
    div=(prec+rec)
    if div!=0:
        return 2*prec*rec/div
    else:
        return 0

Training loop.

In [621]:
classifier = classifier.to(args.device)

loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine', 
                        total=args.num_epochs,
                        position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
                    total=dataset.get_num_batches(args.batch_size), 
                    position=1, 
                    leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_score = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].long())

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the score
            score_t = compute_score(batch_dict['y_target'], y_pred)
            running_score += (score_t - running_score) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, 
                                  score=running_score, 
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_score'].append(running_score)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_score = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].long())

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the score
            #print(prec_(batch_dict['y_target'], y_pred))
            score_t = compute_score(batch_dict['y_target'], y_pred)
            running_score += (score_t - running_score) / (batch_index + 1)
            
            val_bar.set_postfix(loss=running_loss, 
                                score=running_score, 
                                epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_score'].append(running_score)
       
        scheduler.step(train_state['val_loss'][-1])

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
        
except KeyboardInterrupt:
    print("Exiting loop")


training routine:   0%|          | 0/30 [00:00<?, ?it/s]

split=train:   0%|          | 0/41 [00:00<?, ?it/s]

split=val:   0%|          | 0/17 [00:00<?, ?it/s]

In [622]:
train_state['val_score']

[0.0,
 0.38571786463150237,
 0.3785239620119938,
 0.6853752515464574,
 0.6622662408696478,
 0.7343274310651297,
 0.7469441251243459,
 0.7220251421271727,
 0.7613986486868969,
 0.7517713073479426,
 0.7586485139622124,
 0.7532694206741706,
 0.7339676334027201,
 0.7598604250297295,
 0.7627357478267159,
 0.7597114833987981,
 0.7627883089535341,
 0.7617852697446603,
 0.7630845183584016,
 0.7590623085455817,
 0.7581127562174875,
 0.7623577775482678,
 0.7663495672192375,
 0.7616477787654382,
 0.7662441533915714,
 0.7609197375700202,
 0.7631577775146676,
 0.7647830071963077,
 0.7605370240235659,
 0.7611922004723347]